# Configuration Section - Important

In [5]:
colab_flag = False  # Set it true if the notebook is run on Colab
language_model ="fin_lab_large"  # imdb | abc | fin_unlabelled | fin_lab_balance | fin_lab_large
set_labels = "large"

# Importing Modules

## General Libraries

In [6]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

## For Colab

In [7]:
if colab_flag:
  
  !pip install -Uqq fastbook   
  from fastbook import * 
  from google.colab import drive 
  drive.mount('/content/drive')
  %cd /content/drive/My\ Drive/Colab\ Notebooks/

  root = "/content/drive/My Drive/Colab Notebooks/"

  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)

else:

  root = "C:/Users/user/ML/Ironhack/GitHub/Final_Project/"

## Deep Learning

In [8]:
from fastai.text.all import *

# Loading Data

In [9]:
with open(f"Data/dls/dls_clas_{set_labels}", "rb") as f:
    
    dls_clas = pickle.load(f)

    f.close()

    inferer = f"{language_model}_{set_labels}_inferer"

# Loading the LSTM model

In [10]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy).to_fp16()

C:\Users\user\miniconda3\envs\DA_ENV\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [11]:
learn.load(inferer)

# Predicting the test set

In [12]:
fin_lab_large_df = pd.read_csv("Data/csv/fin_lab_large.csv")
fin_lab_large_df["prediction"]=fin_lab_large_df["prob_pos"] = fin_lab_large_df["prob_neg"] = fin_lab_large_df["certainty"] = None
fin_lab_large_df

,text,sentiment,set,prediction,prob_pos,prob_neg,certainty
0,Operating cash flow after investments totalled EUR 61.3 mn compared to EUR 75.3 mn in the corresponding period in 2007 .,negative,train,None,None,None,None
1,"TietoEnator was down 1.13 pct to 18.38 , extending recent lows after last week 's second-quarter report , dealers said .",negative,train,None,None,None,None
2,"In Finland 's Hobby Hall 's sales decreased by 10 % , and international sales fell by 19 % .",negative,train,None,None,None,None
3,"Svyturys-Utenos Alus , which is controlled by the Nordic group Baltic Beverages Holding ( BBH ) , posted a 4.7-per-cent growth in beer sales for January-May to 46.22 million litres .",positive,train,None,None,None,None
4,"`` Lining stone sales were also good in the early autumn , and order books are strong to the end of the year .",positive,train,None,None,None,None
...,...,...,...,...,...,...,...
1940,"The chain posted sales of 298 million euros for full 2005 , a rise of 19.5 percent , year-on-year .",positive,test,None,None,None,None
1941,Satama earned Data Management Solutions competency with Business Intelligence specialization recentlyvia the acquisition of Fimentor Oy .,positive,test,None,None,None,None
1942,"Rinkuskiai raised the sales by 18.1 percent , to 1.37 million liters , while the sales of Kauno Alus grew by 14.3 percent , to 960,000 liters .",positive,test,None,None,None,None
1943,"Earnings per share ( EPS ) in the first half of 2007 amounted to EUR0 .29 , down from EUR0 .40 year ago .",negative,test,None,None,None,None


In [13]:
pred = learn.predict(fin_lab_large_df["text"][0])
pred

C:\Users\user\miniconda3\envs\DA_ENV\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


('negative', TensorText(0), TensorText([0.5896, 0.4104]))

In [14]:
for row in tqdm(range(0, len(fin_lab_large_df))):

  pred = learn.predict(fin_lab_large_df["text"][row])
  fin_lab_large_df["prediction"][row] = pred[0]
  fin_lab_large_df["prob_pos"][row] = float(pred[2][1])
  fin_lab_large_df["prob_neg"][row] = float(pred[2][0])
  fin_lab_large_df["certainty"][row] = abs(fin_lab_large_df["prob_pos"][row]-fin_lab_large_df["prob_neg"][row])

  0%|          | 0/1945 [00:00<?, ?it/s]C:\Users\user\miniconda3\envs\DA_ENV\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


  0%|          | 1/1945 [00:00<03:57,  8.18it/s]C:\Users\user\miniconda3\envs\DA_ENV\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


  0%|          | 2/1945 [00:00<03:51,  8.38it/s]C:\Users\user\miniconda3\envs\DA_ENV\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


  0%|          | 3/1945 [00:00<03:52,  8.37it/s]C:\Users\user\miniconda3\envs\DA_ENV\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


  0%|          | 4/1945 [00:00<04:24,  7.33it/s]C:\Users\user\miniconda3\envs\DA_ENV\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


...

100%|█████████▉| 1941/1945 [04:25<00:00,  7.69it/s]C:\Users\user\miniconda3\envs\DA_ENV\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


100%|█████████▉| 1942/1945 [04:25<00:00,  7.43it/s]C:\Users\user\miniconda3\envs\DA_ENV\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


100%|█████████▉| 1943/1945 [04:25<00:00,  7.57it/s]C:\Users\user\miniconda3\envs\DA_ENV\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


100%|█████████▉| 1944/1945 [04:26<00:00,  7.35it/s]C:\Users\user\miniconda3\envs\DA_ENV\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


100%|██████████| 1945/1945 [04:26<00:00,  7.31it/s]


In [19]:
fin_lab_large_df

,text,sentiment,set,prediction,prob_pos,prob_neg,certainty
0,Operating cash flow after investments totalled EUR 61.3 mn compared to EUR 75.3 mn in the corresponding period in 2007 .,negative,train,negative,0.28887,0.71113,0.42226
1,"TietoEnator was down 1.13 pct to 18.38 , extending recent lows after last week 's second-quarter report , dealers said .",negative,train,negative,0.0036571,0.996343,0.992686
2,"In Finland 's Hobby Hall 's sales decreased by 10 % , and international sales fell by 19 % .",negative,train,negative,0.0167211,0.983279,0.966558
3,"Svyturys-Utenos Alus , which is controlled by the Nordic group Baltic Beverages Holding ( BBH ) , posted a 4.7-per-cent growth in beer sales for January-May to 46.22 million litres .",positive,train,positive,0.959269,0.0407311,0.918538
4,"`` Lining stone sales were also good in the early autumn , and order books are strong to the end of the year .",positive,train,positive,0.967564,0.0324362,0.935128
...,...,...,...,...,...,...,...
1940,"The chain posted sales of 298 million euros for full 2005 , a rise of 19.5 percent , year-on-year .",positive,test,positive,0.969928,0.0300718,0.939856
1941,Satama earned Data Management Solutions competency with Business Intelligence specialization recentlyvia the acquisition of Fimentor Oy .,positive,test,positive,0.989119,0.0108813,0.978237
1942,"Rinkuskiai raised the sales by 18.1 percent , to 1.37 million liters , while the sales of Kauno Alus grew by 14.3 percent , to 960,000 liters .",positive,test,positive,0.995444,0.00455611,0.990888
1943,"Earnings per share ( EPS ) in the first half of 2007 amounted to EUR0 .29 , down from EUR0 .40 year ago .",negative,test,negative,0.015937,0.984063,0.968126


In [21]:
fin_lab_large_df.to_csv("Data/csv/fin_lab_large_pred.csv")

In [22]:
# Let see on wich ones the model has failed

fin_lab_large_df[fin_lab_large_df["sentiment"] != fin_lab_large_df["prediction"]]

,text,sentiment,set,prediction,prob_pos,prob_neg,certainty
61,"The largest restructuring is planned for Italy , where negotiations on about 40 job cuts are to be initiated today , the company noted .",negative,train,positive,0.777975,0.222025,0.555951
76,"Finnish power supply solutions and systems provider Efore Oyj said its net loss widened to 3.2 mln euro $ 4.2 mln for the first quarter of fiscal 2006-2007 ending October 31 , 2007 from 900,000 euro $ 1.2 mln for the same period of fiscal 2005-06 .",negative,train,positive,0.642461,0.357539,0.284921
115,Profit for the period was EUR 15.6 mn compared to EUR 14.1 mn in 2007 .,positive,train,negative,0.446163,0.553837,0.107674
163,"( ADPnews ) - Oct 21 , 2009 - Finland-based IT consultancy Tieto Oyj ( HEL : TIE1V ) said today its net profit plunged to EUR 29.4 million ( USD 43.9 m ) for the first nine months of 2009 from EUR 58.7 million for the same period o",negative,train,positive,0.973317,0.0266831,0.946634
176,Nokia Siemens Networks has struggled to make a profit in the past two years .,negative,train,positive,0.819533,0.180467,0.639065
...,...,...,...,...,...,...,...
1880,The program 's target is structural cost reductions of about EUR 30mn in 2009 .,positive,test,negative,0.234845,0.765155,0.53031
1885,"Operating profit was EUR 9.8 mn , compared to a loss of EUR 12.7 mn in the corresponding period in 2009 .",positive,test,negative,0.493866,0.506134,0.0122674
1891,The new factory working model and reorganisations would decrease Nokian Tyres ' costs in the factory by EUR 30 million ( USD 38.7 m ) .,positive,test,negative,0.440951,0.559049,0.118099
1919,"Therefore , the company 's 2005 result will remain weaker than that of 2004 .",negative,test,positive,0.829527,0.170473,0.659053


# Using the model in new data


In [ ]:
guardian_df = pd.read_csv("Data/csv/guardian_headlines.csv")
guardian_df

,Time,Headlines
0,18-Jul-20,Johnson is asking Santa for a Christmas recovery
1,18-Jul-20,‘I now fear the worst’: four grim tales of working life upended by Covid-19
2,18-Jul-20,Five key areas Sunak must tackle to serve up economic recovery
3,18-Jul-20,Covid-19 leaves firms ‘fatally ill-prepared’ for no-deal Brexit
4,18-Jul-20,The Week in Patriarchy \n\n\n Bacardi's 'lady vodka': the latest in a long line of depressing gendered products
...,...,...
17795,17-Dec-17,How investing in solar energy can create a brighter future for Africa
17796,17-Dec-17,Poundland suppliers hit by insurance downgrade
17797,17-Dec-17,Cryptocurrencies: City watchdog to investigate new fundraising trend
17798,17-Dec-17,Unilever sells household name spreads to KKR for £6bn


In [ ]:
guardian_df = (pd.DataFrame(guardian_df["Headlines"]).rename({"Headlines": "text"}, axis=1)
                 .dropna()
                 .reset_index(drop=True) 
               )
guardian_df["prob_pos"] = guardian_df["prob_neg"] = guardian_df["certainty"] = guardian_df["sentiment"] = None
guardian_df

,text,prob_pos,prob_neg,certainty,sentiment
0,Johnson is asking Santa for a Christmas recovery,None,None,None,None
1,‘I now fear the worst’: four grim tales of working life upended by Covid-19,None,None,None,None
2,Five key areas Sunak must tackle to serve up economic recovery,None,None,None,None
3,Covid-19 leaves firms ‘fatally ill-prepared’ for no-deal Brexit,None,None,None,None
4,The Week in Patriarchy \n\n\n Bacardi's 'lady vodka': the latest in a long line of depressing gendered products,None,None,None,None
...,...,...,...,...,...
17795,How investing in solar energy can create a brighter future for Africa,None,None,None,None
17796,Poundland suppliers hit by insurance downgrade,None,None,None,None
17797,Cryptocurrencies: City watchdog to investigate new fundraising trend,None,None,None,None
17798,Unilever sells household name spreads to KKR for £6bn,None,None,None,None


In [ ]:
learn.predict(guardian_df["text"][0])

('negative', TensorText(0), TensorText([0.5471, 0.4529]))

In [ ]:
for row in tqdm(range(5000, 10000)):

  pred = learn.predict(guardian_df["text"][row])
  guardian_df["sentiment"][row] = pred[0]
  guardian_df["prob_pos"][row] = float(pred[2][1])
  guardian_df["prob_neg"][row] = float(pred[2][0])
  guardian_df["certainty"][row] = abs(guardian_df["prob_pos"][row]-guardian_df["prob_neg"][row])

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 3/5000 [00:00<03:20, 24.95it/s]

  0%|          | 6/5000 [00:00<03:30, 23.77it/s]

  0%|          | 9/5000 [00:00<03:22, 24.69it/s]

  0%|          | 12/5000 [00:00<03:24, 24.42it/s]

...

100%|█████████▉| 4989/5000 [03:23<00:00, 25.52it/s]

100%|█████████▉| 4992/5000 [03:23<00:00, 25.45it/s]

100%|█████████▉| 4995/5000 [03:23<00:00, 25.57it/s]

100%|█████████▉| 4998/5000 [03:24<00:00, 23.24it/s]

100%|██████████| 5000/5000 [03:24<00:00, 24.49it/s]


In [ ]:
for row in tqdm(range(10000, len(guardian_df))):

  pred = learn.predict(guardian_df["text"][row])
  guardian_df["sentiment"][row] = pred[0]
  guardian_df["prob_pos"][row] = float(pred[2][1])
  guardian_df["prob_neg"][row] = float(pred[2][0])
  guardian_df["certainty"][row] = abs(guardian_df["prob_pos"][row]-guardian_df["prob_neg"][row])

  0%|          | 0/7800 [00:00<?, ?it/s]

  0%|          | 3/7800 [00:00<05:13, 24.89it/s]

  0%|          | 6/7800 [00:00<05:41, 22.85it/s]

  0%|          | 9/7800 [00:00<05:32, 23.42it/s]

  0%|          | 12/7800 [00:00<05:34, 23.29it/s]

...

100%|█████████▉| 7788/7800 [05:12<00:00, 24.09it/s]

100%|█████████▉| 7791/7800 [05:12<00:00, 24.41it/s]

100%|█████████▉| 7794/7800 [05:12<00:00, 24.02it/s]

100%|█████████▉| 7797/7800 [05:12<00:00, 24.31it/s]

100%|██████████| 7800/7800 [05:12<00:00, 24.93it/s]


In [ ]:
guardian_df.to_csv("Data/csv/guardian_headlines_labeled.csv", index=False)

In [ ]:
guardian_df

,text,prob_pos,prob_neg,certainty,sentiment
0,Johnson is asking Santa for a Christmas recovery,0.452906,0.547094,0.0941872,negative
1,‘I now fear the worst’: four grim tales of working life upended by Covid-19,0.18279,0.81721,0.63442,negative
2,Five key areas Sunak must tackle to serve up economic recovery,0.650689,0.349311,0.301379,positive
3,Covid-19 leaves firms ‘fatally ill-prepared’ for no-deal Brexit,0.634331,0.365669,0.268662,positive
4,The Week in Patriarchy \n\n\n Bacardi's 'lady vodka': the latest in a long line of depressing gendered products,0.974264,0.0257361,0.948528,positive
...,...,...,...,...,...
17795,How investing in solar energy can create a brighter future for Africa,0.987204,0.0127965,0.974407,positive
17796,Poundland suppliers hit by insurance downgrade,0.0681024,0.931898,0.863795,negative
17797,Cryptocurrencies: City watchdog to investigate new fundraising trend,0.964288,0.0357122,0.928576,positive
17798,Unilever sells household name spreads to KKR for £6bn,0.877687,0.122313,0.755373,positive
